### Adjust Word Level

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


#### While Loop

In [3]:
lang_folder = "Turkish"
lang_pair = "Arabic"

In [4]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
164294,buldugumuzda,43
164295,boşandıklarını,42
164296,endişenlenme,42
164297,uzaylıymışım,42


In [5]:
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence_All.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext.
df_twogram_sent

,twogram,frequency
0,teşekkür ederim,244149
1,öyle mi,209900
2,ne oldu,195799
3,aman tanrım,189521
4,özür dilerim,153784
...,...,...
1036515,güzeldi tommy,3
1036516,durumu tuhaflaştırma,3
1036517,güzeldi canım,3
1036518,güzeldi daniel,3


In [6]:
lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
lang_pair_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish_And_Arabic (ara)_Etymologeek_Result_All.xlsx']

In [7]:
df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair

,search_word,dict_entry_main,language_main,definition_main,dict_entry,language,definition
0,mı,mıknatıs,Turkish,(Noun) Magnet.Magnet.,مِغْنَاطِيس,Arabic (ara),NaN
1,mı,mısdak,Turkish,(Noun) (obsolete) criterion.(obsolete) criterion.,مِصْدَاق,Arabic (ara),NaN
2,şey,şey,Turkish,(Noun Interjection) (slang) genital organ. Thi...,شَيْئاً,Arabic (ara),NaN
3,şey,şeytan,Turkish,(Noun) Devil.Devil.,شَيَّطَ,Arabic (ara),NaN
4,şey,şeytanî,Turkish,(Adjective) Devilish.Devilish.,شَيْطَانِيّ,Arabic (ara),NaN
...,...,...,...,...,...,...,...
362,avra,avrat,Turkish,(Noun) Wife.Wife.,عَوِرَ,Arabic (ara),NaN
363,molla,molla,Turkish,NaN,مَوْلًى,Arabic (ara),NaN
364,ruba,rubai,Turkish,"(Noun) Rubai; quatrain in classical Arabic, Pe...",رباع,Arabic (ara),"Four at a time, in a group of four, in fours."
365,كتاب,كتاب,Turkish,NaN,كتاب,Arabic (ara),"(obsolete). Book. Letter, note, paper, piece o..."


In [8]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [9]:
def word_count_result(df,column_list): # df is dataframe, bigger_than is integer
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list)
    df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [10]:
def word_count_bool(df, word_thresh_num, column_list): # df is a dataframe, bigger_than is an integer
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list)
    df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"count"}, inplace=True)
    
    return  df_word_count["count"][df_word_count.loc[:,"count"] > word_thresh_num].any()  

In [19]:
# while loop code block
twogram_num = 100  # 6*10 min.
twogram_pair_num = 100  # 3*10 min.
word_thresh_num = 7

word_start = 0  # 0
word_end = 10  # 10
step_num = 10  # 10
word_limit = 200  # 200
part_num = 1
while word_end <= word_limit:
    df_word = df_word_all.iloc[word_start:word_end,]
    df_word.reset_index(drop=True, inplace=True)

    # language pair twogram
    ngram_list = []
    for i in df_pair["dict_entry_main"]:
        for j in word_tokenize(i):
            for k in df_word["word"]:
                twogram_1_2 = f"{j} {k}"
                ngram_list.append(twogram_1_2)
                twogram_2_1 = f"{k} {j}"
                ngram_list.append(twogram_2_1)
    df_pair_ngram = pd.DataFrame(ngram_list)
    df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)
    df_pair_ngram.iloc[:,0] = df_pair_ngram.iloc[:,0].apply(lambda x: remove_repetition(x))
    df_pair_ngram.drop_duplicates(inplace=True)
    df_pair_ngram.reset_index(drop=True, inplace=True)
    df_lang_pair_twogram = pd.merge(df_twogram_sent, df_pair_ngram, how="inner", on="twogram")
    df_lang_pair_twogram.rename(columns={"twogram":"twogram_pair"}, inplace=True)
    df_lang_pair_twogram.drop_duplicates(inplace=True)
    #df_lang_pair_twogram = df_lang_pair_twogram.head(100)

    # twogram
    word_list = df_word["word"].values.tolist()
    data_kind = "twogram"
    twogram_list  = df_twogram_sent.iloc[:,0].values.tolist()
    
    resultlist2 = []

    manager = multiprocessing.Manager()
    resultlist2 = manager.list()
    
    def word_in_wordgroup2(list_var2):
        mergelist = []
        try:
            word = list_var2.split()
        except:
            pass
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist2.append(list_var2)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup2, twogram_list) # string_word liste 

    result_list2 = list(resultlist2)
    df_result2 = pd.DataFrame(result_list2)
    df_result2 = df_result2.rename(columns = {0: f"{data_kind}"})
    df_result2.iloc[:,0] = df_result2.iloc[:,0].apply(lambda x: remove_repetition(x)) # **
    df_merge2 = pd.merge(df_result2, df_twogram_sent, how="inner", on=f"{data_kind}")
    df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
    df_merge_result2.drop_duplicates(inplace=True)
    df_merge_result2.reset_index(drop=True, inplace=True)
    df_twogram_result = df_merge_result2
    #df_twogram_result = df_twogram_result.head(100)

    # output
    df_output_result = pd.concat([df_word, df_twogram_result, df_lang_pair_twogram], axis=1)

    df_lesson_result = pd.DataFrame(columns=["word","freq_word","twogram","freq_twogram","twogram_pair","freq_twogram_pair"])
    a = 0
    b = 0

    for i in range(0,110):
        # Insert words and their count 
        try:
            word = df_output_result.iloc[i,0]  # word
            freq_word = df_output_result.iloc[i,1]  # word freq
            df_lesson_result.loc[i,"word"] = word
            df_lesson_result.loc[i,"freq_word"] = freq_word
        except:
            pass
         
        # Insert n grams
        try:
            var1 = df_output_result.iloc[a,2]
            freq_var1 = df_output_result.iloc[a,3]
            if (len(df_lesson_result["twogram"]) < twogram_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,"twogram"] = var1
                df_lesson_result.loc[i,"freq_twogram"] = freq_var1
                try:
                    while word_count_bool(df_lesson_result, (word_thresh_num-1), ["twogram"]): # word count result                
                        a += 1
                        var1 = df_output_result.iloc[a,2]
                        freq_var1 = df_output_result.iloc[a,3]
                        df_lesson_result.loc[i,"twogram"] = var1
                        df_lesson_result.loc[i,"freq_twogram"] = freq_var1
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,"twogram"] = np.nan
                    df_lesson_result.loc[i,"freq_twogram"] = np.nan
            else:
                pass
        except:
            pass
        a += 1

        try:
            var2 = df_output_result.iloc[b,4]
            freq_var2 = df_output_result.iloc[b,5]
            if (len(df_lesson_result["twogram_pair"]) < twogram_pair_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,"twogram_pair"] = var2
                df_lesson_result.loc[i,"freq_twogram_pair"] = freq_var2
                try:
                    while word_count_bool(df_lesson_result, (word_thresh_num-4), ["twogram_pair"]): # word count result                
                        b += 1
                        var2 = df_output_result.iloc[b,4]
                        freq_var2 = df_output_result.iloc[b,5]
                        df_lesson_result.loc[i,"twogram_pair"] = var2
                        df_lesson_result.loc[i,"freq_twogram_pair"] = freq_var2
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,"twogram_pair"] = np.nan
                    df_lesson_result.loc[i,"freq_twogram_pair"] = np.nan
            else:
                pass
        except:
            pass
        b += 1

    df_lesson_word_count = word_count_result(df_lesson_result, ["twogram","twogram_pair"])
    df_lesson_result = pd.merge(df_lesson_result, df_lesson_word_count, how="left", on="word")
    df_lesson_result = df_lesson_result.drop_duplicates()
    df_lesson_result.to_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result{part_num}.xlsx", index=False)

    word_start += step_num
    word_end += step_num
    part_num += 1 

In [20]:
df_lesson_result

,word,freq_word,twogram,freq_twogram,twogram_pair,freq_twogram_pair,word_count
0,onları,575868.0,üzgünüm bayan,689.0,hayır üzgünüm,2696,3.0
1,size,570227.0,tabii bayan,133.0,hayır bayan,1816,5.0
2,üzgünüm,567153.0,tabii gel,59.0,tabii elbette,1206,7.0
3,diye,562704.0,küçük bayan,45.0,üzgünüm evlat,1140,1.0
4,gel,561929.0,bayan üzgünüm,31.0,merhaba bayan,820,7.0
5,küçük,561836.0,gel tabii,20.0,tamam üzgünüm,730,6.0
6,teşekkür,541288.0,tabii üzgünüm,10.0,lütfen gel,723,3.0
7,devam,539682.0,size teşekkür,6.0,dünya küçük,571,4.0
8,bayan,538490.0,gel devam,6.0,tamam gel,503,8.0
9,tabii,536683.0,size bayan,4.0,bayan lütfen,419,7.0


In [15]:
word_count_result(df_lesson_result, ["twogram"])

,word,word_count
0,ben,6
1,bir,6
2,bu,6
3,mi,6
4,ne,6
5,ve,6
6,çok,6
7,ama,5
8,de,5
9,için,2


In [16]:
word_count_result(df_lesson_result, ["twogram_pair"])

,word,word_count
0,şey,3
1,için,3
2,ama,3
3,çok,3
4,ve,3
5,ben,3
6,bir,3
7,bu,3
8,ne,3
9,de,3
